# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [6]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.8
Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio server token: 5ouZC...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: relaxed_jones
Kleio version requested: latest
Kleio server version: 12.5.570 (2024-02-29 07:39:03)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [7]:
tlnb.table_row_count_df()

,table,count
0,acts,40
1,attributes,15297
2,class_attributes,63
3,classes,13
4,entities,18644
5,geoentities,359
6,goods,0
7,kleiofiles,5
8,objects,0
9,persons,1066


### Show the kleio files available 

In [8]:
kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,kleio/auc-alunos.cli,auc-alunos.cli,2024-05-01 11:41:49.935741+00:00,V,2024-05-01 11:41:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-01 11:43:45.064480,/rest/reports/kleio/auc-alunos.rpt,/rest/exports/kleio/auc-alunos.xml
1,kleio/b1685.cli,b1685.cli,2024-05-01 11:39:30.037059+00:00,W,2024-05-01 11:39:00+00:00,0,1,I,0,0,No errors,No warnings,2024-05-01 11:44:23.695609,/rest/reports/kleio/b1685.rpt,/rest/exports/kleio/b1685.xml
2,kleio/dehergne-a.cli,dehergne-a.cli,2024-05-01 11:39:38.390269+00:00,V,2024-05-01 11:39:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-01 11:43:59.863426,/rest/reports/kleio/dehergne-a.rpt,/rest/exports/kleio/dehergne-a.xml
3,kleio/dehergne-locations-1644.cli,dehergne-locations-1644.cli,2024-05-01 11:39:35.383210+00:00,V,2024-05-01 11:39:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-01 11:44:09.681128,/rest/reports/kleio/dehergne-locations-1644.rpt,/rest/exports/kleio/dehergne-locations-1644.xml
4,kleio/dehergne-locations-1701.cli,dehergne-locations-1701.cli,2024-05-01 11:39:39.455607+00:00,V,2024-05-01 11:39:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-01 11:44:16.637977,/rest/reports/kleio/dehergne-locations-1701.rpt,/rest/exports/kleio/dehergne-locations-1701.xml


## Optionally clean translations 

In [9]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['kleio/dehergne-locations-1701.cli',
 'kleio/dehergne-locations-1644.cli',
 'kleio/dehergne-a.cli',
 'kleio/b1685.cli',
 'kleio/auc-alunos.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [10]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

INFO:root:Request translation of T kleio/auc-alunos.cli
INFO:root:Request translation of T kleio/b1685.cli
INFO:root:Request translation of T kleio/dehergne-a.cli
INFO:root:Request translation of T kleio/dehergne-locations-1644.cli
INFO:root:Request translation of T kleio/dehergne-locations-1701.cli


ReadTimeout: HTTPConnectionPool(host='127.0.0.1', port=8090): Read timed out. (read timeout=60)

## Check the status of the files

Check the import status of the translated files

I: Imported

E: Imported with error

W: Imported with warnings no errors

N: Not imported

U: Translation updated need to reimport

In [ ]:
imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,auc-alunos.cli,0,0,kleio/auc-alunos.cli
1,I,0,0,b1685.cli,0,1,kleio/b1685.cli
2,I,0,0,dehergne-a.cli,0,0,kleio/dehergne-a.cli
3,I,0,0,dehergne-locations-1644.cli,0,0,kleio/dehergne-locations-1644.cli
4,I,0,0,dehergne-locations-1701.cli,0,0,kleio/dehergne-locations-1701.cli


### Check a translation report

If needed to check for errors.


In [4]:
file_number=2
rpt = tlnb.get_translation_report(imported_files_df, file_number)
print(rpt)


NameError: name 'imported_files_df' is not defined

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [ ]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

No errors
No warnings
